In [1]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import torch
import warnings
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline, BitsAndBytesConfig
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [4]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [5]:
# Evaluator
summary_evaluator = SummaryEvaluator()

In [6]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
t5_pipe = pipeline(
    "summarization",
    model=t5model,
    tokenizer=t5tokenizer,
    max_length=1000,
    truncation=True,
    device=0
)
#t5_pipe.model.config.pad_token_id = t5_pipe.model.config.eos_token_id

In [9]:
t5_llm_lc = HuggingFacePipeline(pipeline=t5_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [10]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """summarize: {context}"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | t5_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        example_query = sentences[-1]
        #summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        #summary = summary_w_prompt.split("summarize: ",1)[1]
        results.append(summary)
        i += 1
    return results

In [11]:
references = [example['summary'] for example in dataset]

In [12]:
import re

In [13]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 1000, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)


1


  0%|          | 1/200 [00:20<1:09:34, 20.98s/it]Your max_length is set to 1000, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


2


  1%|          | 2/200 [00:37<1:00:44, 18.41s/it]Your max_length is set to 1000, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


...

 99%|█████████▉| 198/200 [53:50<00:35, 17.74s/it]Your max_length is set to 1000, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


199


100%|█████████▉| 199/200 [54:02<00:15, 15.89s/it]Your max_length is set to 1000, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


200


100%|██████████| 200/200 [54:13<00:00, 16.27s/it]


In [16]:
print("chunk_size=200, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.16286008619391562, 'rouge2': 0.019116605348640223, 'rougeL': 0.11466688575036507}
Average BLEU score: 0.014923957408900401


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4671219289302826
Average VECTOR_SIMILARITY score: 0.3644565939903259
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.16903515100276909, 'rouge2': 0.021214850226273382, 'rougeL': 0.12095900186234072}
Average BLEU score: 0.015235116292269469


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47084495425224304
Average VECTOR_SIMILARITY score: 0.381409227848053
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.17044380922620125, 'rouge2': 0.019778977274572184, 'rougeL': 0.11975367339408562}
Average BLEU score: 0.01462541423260322


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47295647859573364
Average VECTOR_SIMILARITY score: 0.38887009024620056
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.17066372738235608, 'rouge2': 0.020258583971359583, 'rougeL': 0.1205830608862431}
Average BLEU score: 0.014652664541089634


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.473856121301651
Average VECTOR_SIMILARITY score: 0.39037376642227173


{'rouge': {'rouge1': 0.17066372738235608,
  'rouge2': 0.020258583971359583,
  'rougeL': 0.1205830608862431},
 'bleu': 0.014652664541089634,
 'bertscore': 0.473856121301651,
 'vector_similarity': 0.39037376642227173}

In [19]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Your max_length is set to 1000, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)


1


  0%|          | 1/200 [00:12<40:36, 12.24s/it]Your max_length is set to 1000, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


2


  1%|          | 2/200 [00:27<46:04, 13.96s/it]Your max_length is set to 1000, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


...

 99%|█████████▉| 198/200 [54:24<00:32, 16.35s/it]Your max_length is set to 1000, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


199


100%|█████████▉| 199/200 [54:35<00:14, 14.94s/it]Your max_length is set to 1000, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


200


100%|██████████| 200/200 [54:46<00:00, 16.43s/it]


In [20]:
print("chunk_size=200, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1617739983021751, 'rouge2': 0.02068809180808556, 'rougeL': 0.1136142904727035}
Average BLEU score: 0.015491385603238482


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4659484922885895
Average VECTOR_SIMILARITY score: 0.3662678301334381
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.1683287609952646, 'rouge2': 0.021336200173819587, 'rougeL': 0.12129781521833755}
Average BLEU score: 0.015410473639452036


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.469826340675354
Average VECTOR_SIMILARITY score: 0.3833193778991699
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.1649467443115041, 'rouge2': 0.01875935434002417, 'rougeL': 0.11677560714726304}
Average BLEU score: 0.014365066991526669


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.470882773399353
Average VECTOR_SIMILARITY score: 0.39098480343818665
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.16690097871280143, 'rouge2': 0.01925173740780598, 'rougeL': 0.11769940758124477}
Average BLEU score: 0.01457466618479571


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4715613126754761
Average VECTOR_SIMILARITY score: 0.39306017756462097


{'rouge': {'rouge1': 0.16690097871280143,
  'rouge2': 0.01925173740780598,
  'rougeL': 0.11769940758124477},
 'bleu': 0.01457466618479571,
 'bertscore': 0.4715613126754761,
 'vector_similarity': 0.39306017756462097}

In [21]:
rag_model = RAG_Summarization_Model(chunk_size=150, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Your max_length is set to 1000, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


1


  0%|          | 1/200 [00:12<42:34, 12.84s/it]Your max_length is set to 1000, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


2


  1%|          | 2/200 [00:27<46:41, 14.15s/it]Your max_length is set to 1000, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


...

 99%|█████████▉| 198/200 [53:23<00:32, 16.10s/it]Your max_length is set to 1000, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


199


100%|█████████▉| 199/200 [53:37<00:15, 15.46s/it]Your max_length is set to 1000, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


200


100%|██████████| 200/200 [53:47<00:00, 16.14s/it]


In [22]:
print("chunk_size=150, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=150, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1570224814555856, 'rouge2': 0.019136085679797344, 'rougeL': 0.11503432464019504}
Average BLEU score: 0.013951261321493347


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4614761769771576
Average VECTOR_SIMILARITY score: 0.35941824316978455
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.1654130611946686, 'rouge2': 0.022039015952369723, 'rougeL': 0.11821381601484095}
Average BLEU score: 0.014964198664245996


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.46826648712158203
Average VECTOR_SIMILARITY score: 0.3811994194984436
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.16722340928863255, 'rouge2': 0.023599109512288878, 'rougeL': 0.11995484186697648}
Average BLEU score: 0.015018538874018135


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47054848074913025
Average VECTOR_SIMILARITY score: 0.39501410722732544
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.1664295108672353, 'rouge2': 0.02353107790039556, 'rougeL': 0.12000679124198702}
Average BLEU score: 0.014987915206020844


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4712683856487274
Average VECTOR_SIMILARITY score: 0.40068891644477844


{'rouge': {'rouge1': 0.1664295108672353,
  'rouge2': 0.02353107790039556,
  'rougeL': 0.12000679124198702},
 'bleu': 0.014987915206020844,
 'bertscore': 0.4712683856487274,
 'vector_similarity': 0.40068891644477844}